In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fcc_101_28GHz.csv')

In [3]:
df.columns

Index(['auction_id', 'round', 'item', 'item_description', 'bidding_units',
       'new_bids', 'tied_bids', 'pw_bidder', 'pw_bidder_frn', 'pwb_amount',
       'bidding_credit', 'net_pwb_amount', 'round_of_pwb', 'increments_of_pwb',
       'date_of_pwb', 'previous_pw_bidder', 'previous_pw_bidder_frn',
       'previous_pwb_amount', 'previous_pwb_bidding_credit',
       'previous_net_pwb_amount', 'round_of_previous_pwb', 'item_closed',
       'block', 'population', 'bandwidth', 'market',
       'market_based_small_business_bidding_credit_cap',
       'market_based_rural_bidding_credit_cap'],
      dtype='object')

In [4]:
print(f"Dataset size: {len(df)}")

Dataset size: 540672


In [5]:
print(f"Auction ID: {set(df['auction_id'])}, Rounds: {len(set(df['round']))}, Items: {len(set(df['item']))}, Bidders: {len(set(df['pw_bidder']))}")

Auction ID: {101}, Rounds: 176, Items: 3072, Bidders: 38


In [6]:
df = df.sort_values(by=['round'], ascending=True)

In [7]:
df.iloc[200]

auction_id                                                         101
round                                                                1
item                                                         C20085-L2
item_description                                           Jackson, KS
bidding_units                                                       57
new_bids                                                             1
tied_bids                                                          0.0
pw_bidder                                         T-Mobile License LLC
pw_bidder_frn                                                  1565449
pwb_amount                                                        1100
bidding_credit                                                     0.0
net_pwb_amount                                                    1100
round_of_pwb                                                         1
increments_of_pwb                                                  1.0
date_o

## Auction Implementation

In [8]:
import torch
import torch.nn as nn
import logging

from comblearn.data import DSFValueFunction
from comblearn.env import CombinatorialAuctionWithData

logging.basicConfig(level=20)

In [9]:
device = 'cuda'

def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [30]:
bidders = list(set(df['pw_bidder']))
items = []

dataset = []
value_functions = []
for bidder in bidders:
    ds = df.loc[df['pw_bidder'] == bidder].groupby(['round']).agg({'pwb_amount': 'sum'}).join(pd.get_dummies(df['item']))
    X, y = torch.tensor(ds[ds.columns[1:]].values.tolist()).float(), torch.tensor(ds['pwb_amount'].values.tolist()).float().unsqueeze(1)
    items = list(ds.columns[1:])
    dataset.append((X, y))
    print(X.shape, y.shape, torch.max(y).numpy(), len(items))
    value_functions.append(DSFValueFunction(items, torch.max(y).numpy(), [5, 6, 3], [5000, 3000, 2000]))

torch.Size([176, 3072]) torch.Size([176, 1]) 1030660.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 129404200.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 5714810.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 6233290.0 3072
torch.Size([175, 3072]) torch.Size([175, 1]) 65980.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 59000.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 1040150.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 75430960.0 3072
torch.Size([175, 3072]) torch.Size([175, 1]) 435400.0 3072
torch.Size([175, 3072]) torch.Size([175, 1]) 2336400.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 193190.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 34950.0 3072
torch.Size([173, 3072]) torch.Size([173, 1]) 209300.0 3072
torch.Size([131, 3072]) torch.Size([131, 1]) 24300.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 58000.0 3072
torch.Size([171, 3072]) torch.Size([171, 1]) 1184000.0 3072
torch.Size([176, 3072]) torch.Size([176, 1]) 37495

In [31]:
auction = CombinatorialAuctionWithData(bidders, items, dataset, value_functions)
allocations, payments = auction.run(epochs=1000, lr=0.001, delta=0.005, sample_rate=40)

INFO:root:Allocation calculation...
INFO:root:Bidder 1, loss: 159473991680.0
INFO:root:Bidder 2, loss: 3037593876824064.0
INFO:root:Bidder 3, loss: 12707663183872.0
INFO:root:Bidder 4, loss: 6048503037952.0
INFO:root:Bidder 5, loss: 605673344.0
INFO:root:Bidder 6, loss: 391172960.0
INFO:root:Bidder 7, loss: 437623783424.0
INFO:root:Bidder 8, loss: 1598154780704768.0
INFO:root:Bidder 9, loss: 45894164480.0
INFO:root:Bidder 10, loss: 1924135780352.0
INFO:root:Bidder 11, loss: 2190402560.0
INFO:root:Bidder 12, loss: 401025888.0
INFO:root:Bidder 13, loss: 13405801472.0
INFO:root:Bidder 14, loss: 245941968.0
INFO:root:Bidder 15, loss: 726014784.0
INFO:root:Bidder 16, loss: 246352822272.0
INFO:root:Bidder 17, loss: 118993305600.0
INFO:root:Bidder 18, loss: 32097242710016.0
INFO:root:Bidder 19, loss: 0.0
INFO:root:Bidder 20, loss: 2084692688896.0
INFO:root:Bidder 21, loss: 555981144064.0
INFO:root:Bidder 22, loss: 13665226981376.0
INFO:root:Bidder 23, loss: 11499528192.0
INFO:root:Bidder 24, 